# Market-Neutral Carry Strategy: Stress Test Analysis

This notebook conducts comprehensive stress testing of the strategy under historical crisis scenarios and synthetic shocks.

**Objectives:**
1. Test strategy during 2008 financial crisis
2. Evaluate COVID-19 crash performance
3. Simulate synthetic stress scenarios
4. Analyze correlation breakdown
5. Test risk management effectiveness

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import strategy modules
from data_acquisition import MultiAssetDataAcquisition
from factor_models import FactorModels
from signal_generator import CarrySignalGenerator
from portfolio_constructor import PortfolioConstructor
from backtester import CarryBacktester

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("Libraries imported successfully")

## 1. Load Data and Run Backtest

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Stress test scenarios:")
for scenario in config['risk']['stress_scenarios']:
    print(f"  - {scenario['name']}")

In [ ]:
# Run complete pipeline
print("Running backtest...")
data_acq = MultiAssetDataAcquisition(config)
dataset = data_acq.get_full_dataset()

factor_models = FactorModels(config)
factor_predictions = factor_models.fit_all_models(dataset)

signal_gen = CarrySignalGenerator(config)
signals = signal_gen.generate_all_signals(dataset, factor_predictions)

portfolio_constructor = PortfolioConstructor(config)
all_dates = []
for asset_class, data in dataset.items():
    if isinstance(data, dict):
        for key, df in data.items():
            if isinstance(df, pd.DataFrame) and len(df) > 0:
                all_dates.extend(df.index.tolist())

min_date = min(all_dates)
max_date = max(all_dates)
rebalance_dates = pd.date_range(start=min_date, end=max_date, 
                                freq=config['data']['rebalance_frequency'])

portfolio_history = portfolio_constructor.construct_portfolio_timeseries(
    signals, dataset, rebalance_dates.tolist())

backtester = CarryBacktester(config)
results = backtester.run_backtest(portfolio_history, dataset)

results_df = results['results']
print("Backtest complete")

## 2. Historical Stress Test: 2008 Financial Crisis

In [ ]:
# Extract 2008 crisis period
crisis_start = pd.Timestamp('2008-09-01')
crisis_end = pd.Timestamp('2009-03-31')

if crisis_start in results_df.index and crisis_end in results_df.index:
    crisis_results = results_df.loc[crisis_start:crisis_end]
    
    # Calculate crisis metrics
    crisis_returns = crisis_results['daily_return']
    crisis_total_return = (1 + crisis_returns).prod() - 1
    crisis_vol = crisis_returns.std() * np.sqrt(252)
    crisis_sharpe = crisis_returns.mean() / crisis_returns.std() * np.sqrt(252) if crisis_returns.std() > 0 else 0
    
    # Drawdown
    cumulative = (1 + crisis_returns).cumprod()
    running_max = cumulative.expanding().max()
    crisis_dd = ((cumulative - running_max) / running_max).min()
    
    print("2008 FINANCIAL CRISIS PERFORMANCE")
    print("="*60)
    print(f"Period: {crisis_start.date()} to {crisis_end.date()}")
    print(f"Total Return:    {crisis_total_return:>10.2%}")
    print(f"Volatility:      {crisis_vol:>10.2%}")
    print(f"Sharpe Ratio:    {crisis_sharpe:>10.2f}")
    print(f"Max Drawdown:    {crisis_dd:>10.2%}")
    print(f"Worst Day:       {crisis_returns.min():>10.2%}")
    print(f"Best Day:        {crisis_returns.max():>10.2%}")
else:
    print("2008 crisis period not in backtest range")
    crisis_results = None

In [ ]:
# Plot 2008 crisis performance
if crisis_results is not None:
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    # Equity curve
    axes[0].plot(crisis_results.index, crisis_results['equity'], linewidth=2)
    axes[0].set_title('2008 Crisis: Equity Curve', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Capital ($)')
    axes[0].grid(True, alpha=0.3)
    
    # Daily returns
    axes[1].bar(crisis_results.index, crisis_results['daily_return'] * 100, alpha=0.7)
    axes[1].set_title('2008 Crisis: Daily Returns', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Return (%)')
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1].grid(True, alpha=0.3)
    
    # Drawdown
    cumulative = (1 + crisis_results['daily_return']).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    
    axes[2].fill_between(crisis_results.index, 0, drawdown * 100, alpha=0.3, color='red')
    axes[2].plot(crisis_results.index, drawdown * 100, color='red', linewidth=2)
    axes[2].set_title('2008 Crisis: Drawdown', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Drawdown (%)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 3. Historical Stress Test: 2020 COVID-19 Crash

In [ ]:
# Extract COVID crash period
covid_start = pd.Timestamp('2020-02-15')
covid_end = pd.Timestamp('2020-04-30')

if covid_start in results_df.index and covid_end in results_df.index:
    covid_results = results_df.loc[covid_start:covid_end]
    
    # Calculate metrics
    covid_returns = covid_results['daily_return']
    covid_total_return = (1 + covid_returns).prod() - 1
    covid_vol = covid_returns.std() * np.sqrt(252)
    covid_sharpe = covid_returns.mean() / covid_returns.std() * np.sqrt(252) if covid_returns.std() > 0 else 0
    
    cumulative = (1 + covid_returns).cumprod()
    running_max = cumulative.expanding().max()
    covid_dd = ((cumulative - running_max) / running_max).min()
    
    print("2020 COVID-19 CRASH PERFORMANCE")
    print("="*60)
    print(f"Period: {covid_start.date()} to {covid_end.date()}")
    print(f"Total Return:    {covid_total_return:>10.2%}")
    print(f"Volatility:      {covid_vol:>10.2%}")
    print(f"Sharpe Ratio:    {covid_sharpe:>10.2f}")
    print(f"Max Drawdown:    {covid_dd:>10.2%}")
    print(f"Worst Day:       {covid_returns.min():>10.2%}")
    print(f"Best Day:        {covid_returns.max():>10.2%}")
else:
    print("COVID crash period not in backtest range")
    covid_results = None

In [ ]:
# Plot COVID crash performance
if covid_results is not None:
    fig, axes = plt.subplots(3, 1, figsize=(14, 12))
    
    axes[0].plot(covid_results.index, covid_results['equity'], linewidth=2, color='purple')
    axes[0].set_title('COVID-19 Crash: Equity Curve', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Capital ($)')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].bar(covid_results.index, covid_results['daily_return'] * 100, alpha=0.7, color='purple')
    axes[1].set_title('COVID-19 Crash: Daily Returns', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Return (%)')
    axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1].grid(True, alpha=0.3)
    
    cumulative = (1 + covid_results['daily_return']).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    
    axes[2].fill_between(covid_results.index, 0, drawdown * 100, alpha=0.3, color='purple')
    axes[2].plot(covid_results.index, drawdown * 100, color='purple', linewidth=2)
    axes[2].set_title('COVID-19 Crash: Drawdown', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Drawdown (%)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4. Correlation Breakdown Analysis

In [ ]:
# Analyze correlation with market (SPX) during stress periods
if 'macro_factors' in dataset and 'VIX' in dataset['macro_factors'].columns:
    vix = dataset['macro_factors']['VIX']
    
    # Calculate rolling correlation with VIX
    returns = results_df['daily_return']
    vix_changes = vix.pct_change()
    
    # Align indices
    common_idx = returns.index.intersection(vix_changes.index)
    returns_aligned = returns.loc[common_idx]
    vix_aligned = vix_changes.loc[common_idx]
    
    rolling_corr = returns_aligned.rolling(window=63).corr(vix_aligned)
    
    plt.figure(figsize=(14, 6))
    plt.plot(rolling_corr.index, rolling_corr, linewidth=2)
    plt.axhline(y=config['risk']['max_spx_correlation'], color='r', 
               linestyle='--', label=f"Alert Level: {config['risk']['max_spx_correlation']}")
    plt.axhline(y=-config['risk']['max_spx_correlation'], color='r', linestyle='--')
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.title('Rolling 63-Day Correlation with VIX', fontsize=14, fontweight='bold')
    plt.ylabel('Correlation')
    plt.xlabel('Date')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nCORRELATION WITH VIX")
    print("="*60)
    print(f"Average correlation: {rolling_corr.mean():.3f}")
    print(f"Max positive correlation: {rolling_corr.max():.3f}")
    print(f"Max negative correlation: {rolling_corr.min():.3f}")
    print(f"% time above alert level: {(rolling_corr.abs() > config['risk']['max_spx_correlation']).sum() / len(rolling_corr):.1%}")

## 5. Tail Risk Analysis

In [ ]:
# Analyze tail events
returns = results_df['daily_return'].dropna()

# Define tail thresholds
left_tail = returns.quantile(0.01)
right_tail = returns.quantile(0.99)

print("TAIL RISK ANALYSIS")
print("="*60)
print(f"1% VaR (daily):    {left_tail:>10.2%}")
print(f"5% VaR (daily):    {returns.quantile(0.05):>10.2%}")
print(f"99% quantile:      {right_tail:>10.2%}")
print(f"95% quantile:      {returns.quantile(0.95):>10.2%}")

# Count tail events
left_tail_count = (returns < left_tail).sum()
right_tail_count = (returns > right_tail).sum()

print(f"\nTail event frequency:")
print(f"Large losses (< 1%): {left_tail_count} days")
print(f"Large gains (> 99%): {right_tail_count} days")

In [ ]:
# Plot return distribution with tails highlighted
plt.figure(figsize=(14, 6))
plt.hist(returns * 100, bins=100, alpha=0.7, edgecolor='black')
plt.axvline(x=left_tail * 100, color='r', linestyle='--', linewidth=2, label='1% VaR')
plt.axvline(x=right_tail * 100, color='g', linestyle='--', linewidth=2, label='99% Quantile')
plt.axvline(x=returns.mean() * 100, color='b', linestyle='--', linewidth=2, label='Mean')
plt.title('Return Distribution with Tail Events', fontsize=14, fontweight='bold')
plt.xlabel('Daily Return (%)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Worst Drawdown Periods Analysis

In [ ]:
# Find all significant drawdown periods
returns = results_df['daily_return']
cumulative = (1 + returns).cumprod()
running_max = cumulative.expanding().max()
drawdown = (cumulative - running_max) / running_max

# Identify periods
significant_dd_threshold = -0.05
in_dd = drawdown < significant_dd_threshold
dd_periods = []
start = None

for date, is_dd in in_dd.items():
    if is_dd and start is None:
        start = date
    elif not is_dd and start is not None:
        dd_periods.append({
            'start': start,
            'end': date,
            'min_dd': drawdown.loc[start:date].min(),
            'duration': (date - start).days
        })
        start = None

# Sort by severity
dd_periods = sorted(dd_periods, key=lambda x: x['min_dd'])

print("TOP 5 WORST DRAWDOWN PERIODS")
print("="*60)
for i, period in enumerate(dd_periods[:5]):
    print(f"\n{i+1}. {period['start'].date()} to {period['end'].date()}")
    print(f"   Max Drawdown: {period['min_dd']:.2%}")
    print(f"   Duration: {period['duration']} days")

## 7. Risk Management Effectiveness

In [ ]:
# Evaluate if risk limits would have been triggered
dd_warning = config['risk']['scale_at_drawdown']
dd_critical = config['risk']['extreme_drawdown_limit']

# Check trigger points
warning_triggers = (drawdown < -dd_warning).sum()
critical_triggers = (drawdown < -dd_critical).sum()

print("RISK MANAGEMENT TRIGGER ANALYSIS")
print("="*60)
print(f"Drawdown warning level ({dd_warning:.0%}): {warning_triggers} days")
print(f"Drawdown critical level ({dd_critical:.0%}): {critical_triggers} days")

if warning_triggers > 0:
    print(f"\nWarning would have reduced leverage on {warning_triggers} days")
if critical_triggers > 0:
    print(f"Critical stop would have been hit on {critical_triggers} days")
else:
    print("\nNo critical stops would have been triggered")

## 8. Synthetic Stress Scenarios

In [ ]:
# Simulate synthetic stress scenarios
print("SYNTHETIC STRESS SCENARIOS")
print("="*60)

# Scenario 1: All carry premia compress simultaneously
print("\nScenario 1: Carry Compression")
print("Assumption: All carry spreads halve overnight")
print("Impact: Positions need to be unwound, incurring transaction costs")

# Estimate impact
avg_gross_exposure = portfolio_history.abs().sum(axis=1).mean()
avg_turnover = portfolio_history.diff().abs().sum(axis=1).mean()
avg_cost = 5  # bps

estimated_cost = avg_gross_exposure * avg_cost / 10000
print(f"Estimated immediate loss: {estimated_cost:.2%}")

# Scenario 2: VIX spike to 80
print("\nScenario 2: VIX Spike to 80")
print("Assumption: Volatility quadruples, correlations go to 1")
print("Impact: All positions move in same direction")

normal_vol = returns.std() * np.sqrt(252)
stress_vol = normal_vol * 4
worst_day_stress = stress_vol / np.sqrt(252) * 3  # 3-sigma event

print(f"Normal volatility: {normal_vol:.2%}")
print(f"Stress volatility: {stress_vol:.2%}")
print(f"Estimated worst day: {worst_day_stress:.2%}")

## 9. Comparison: Normal vs Stress Periods

In [ ]:
# Define stress periods (high VIX)
if 'macro_factors' in dataset and 'VIX' in dataset['macro_factors'].columns:
    vix = dataset['macro_factors']['VIX']
    vix_aligned = vix.reindex(results_df.index, method='ffill')
    
    stress_threshold = 30
    stress_periods = vix_aligned > stress_threshold
    normal_periods = vix_aligned <= stress_threshold
    
    stress_returns = returns[stress_periods]
    normal_returns = returns[normal_periods]
    
    print("PERFORMANCE: NORMAL VS STRESS PERIODS")
    print("="*60)
    print(f"\nNormal Periods (VIX <= {stress_threshold}):")
    print(f"  Average return: {normal_returns.mean()*252:.2%} annualized")
    print(f"  Volatility: {normal_returns.std()*np.sqrt(252):.2%}")
    print(f"  Sharpe: {normal_returns.mean()/normal_returns.std()*np.sqrt(252):.2f}")
    
    print(f"\nStress Periods (VIX > {stress_threshold}):")
    print(f"  Average return: {stress_returns.mean()*252:.2%} annualized")
    print(f"  Volatility: {stress_returns.std()*np.sqrt(252):.2%}")
    print(f"  Sharpe: {stress_returns.mean()/stress_returns.std()*np.sqrt(252):.2f}")
    
    # Plot comparison
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    axes[0].hist([normal_returns*100, stress_returns*100], bins=30, 
                label=['Normal', 'Stress'], alpha=0.7)
    axes[0].set_title('Return Distribution: Normal vs Stress')
    axes[0].set_xlabel('Daily Return (%)')
    axes[0].set_ylabel('Frequency')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    comparison_df = pd.DataFrame({
        'Normal': [normal_returns.mean()*252, normal_returns.std()*np.sqrt(252)],
        'Stress': [stress_returns.mean()*252, stress_returns.std()*np.sqrt(252)]
    }, index=['Return', 'Volatility'])
    
    comparison_df.T.plot(kind='bar', ax=axes[1])
    axes[1].set_title('Annualized Metrics: Normal vs Stress')
    axes[1].set_ylabel('Value')
    axes[1].set_xticklabels(['Normal', 'Stress'], rotation=0)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Summary

**Stress Test Results:**

**Historical Crises:**
- 2008 Financial Crisis: [describe performance]
- 2020 COVID Crash: [describe performance]

**Key Findings:**
1. Maximum drawdown during stress: [X]%
2. Correlation breakdown: [observed/not observed]
3. Risk management triggers: [effective/ineffective]
4. Tail risk exposure: [acceptable/concerning]

**Recommendations:**
1. [Based on stress test results]
2. Consider enhanced hedging during VIX > 30
3. Review position sizing during high correlation regimes